In [32]:
import numpy as np

import re
import tqdm
import urllib
from bs4 import BeautifulSoup

import requests

import collections

from astropy import units as u
from astropy.io import ascii
from astropy import table

In [2]:
def int_to_roman(num):
    roman = collections.OrderedDict()
    roman[1000] = "M"
    roman[900] = "CM"
    roman[500] = "D"
    roman[400] = "CD"
    roman[100] = "C"
    roman[90] = "XC"
    roman[50] = "L"
    roman[40] = "XL"
    roman[10] = "X"
    roman[9] = "IX"
    roman[5] = "V"
    roman[4] = "IV"
    roman[1] = "I"

    def roman_num(num):
        for r in roman.keys():
            x, y = divmod(num, r)
            yield roman[r] * x
            num -= (r * x)
            if num > 0:
                roman_num(num)
            else:
                break

    return "".join([a for a in roman_num(num)])

# Test run 

In [3]:
example_url = 'https://koa.ipac.caltech.edu/cgi-bin/bgServices/nph-bgExec?bgApp=/KOA/nph-KOA&instrument_de=deimos&filetype=science&calibassoc=assoc&locstr=and+xix&regSize=5&resolver=ned&radunits=arcmin&spt_obj=spatial&single_multiple=single'

In [4]:
base_url = 'https://koa.ipac.caltech.edu/cgi-bin/bgServices/nph-bgExec'
encoded = urllib.parse.urlencode({'bgApp':'/KOA/nph-KOA',
                        'instrument_de':'deimos',
                        'filetype':'science',
                        'calibassoc':'assoc',
                        'locstr':'And XIX',
                        'regSize':'5',
                        'radunits':'arcmin',
                        'resolver':'ned',
                        'spt_obj':'spatial',
                        'single_multiple':'single'
                        })
url = base_url + '?' + encoded
url

'https://koa.ipac.caltech.edu/cgi-bin/bgServices/nph-bgExec?spt_obj=spatial&locstr=And+XIX&radunits=arcmin&bgApp=%2FKOA%2Fnph-KOA&calibassoc=assoc&regSize=5&resolver=ned&filetype=science&single_multiple=single&instrument_de=deimos'

In [5]:
res = requests.get(url)

In [6]:
soup = BeautifulSoup(res.text, 'html.parser')

In [7]:
sci = [tag for tag in soup.find_all('a', {'target':'koa'}, string=re.compile('.*DEIMOS.*')) if 'Science' in tag.text]
assert len(sci)==1
sci_url = '://'.join(urllib.parse.urlparse(base_url)[:2]) + sci[0].get('href')

calib = [tag for tag in soup.find_all('a', {'target':'koa'}, string=re.compile('.*DEIMOS.*')) if 'Calibration' in tag.text]
assert len(calib)==1
calib_url = '://'.join(urllib.parse.urlparse(base_url)[:2]) + sci[0].get('href')

sci_url

'https://koa.ipac.caltech.edu/workspace/TMP_GMiqMN_9368/KOA/sci_de_9368.tbl'

In [8]:
sci_tab = ascii.read(sci_url)

# Real business 

In [15]:
def target(name, radius, calib=False, verbose=True):
    amin = radius.to(u.arcmin).value
    
    base_url = 'https://koa.ipac.caltech.edu/cgi-bin/bgServices/nph-bgExec'
    encoded = urllib.parse.urlencode({'bgApp':'/KOA/nph-KOA',
                            'instrument_de':'deimos',
                            'filetype':'science',
                            'calibassoc':'assoc',
                            'locstr':name,
                            'regSize': amin,
                            'radunits':'arcmin',
                            'resolver':'ned',
                            'spt_obj':'spatial',
                            'single_multiple':'single'
                            })
    url = base_url + '?' + encoded
    
    if verbose:
        print('search URL:', url)
    
    res = requests.get(url)
    
    soup = BeautifulSoup(res.text, 'html.parser')
    
    sciorcalib = 'Calibration' if calib else 'Science'
    metadata = [tag for tag in soup.find_all('a', {'target':'koa'}, string=re.compile('.*DEIMOS.*')) if sciorcalib in tag.text]
    assert len(metadata)==1
    metadata_url = '://'.join(urllib.parse.urlparse(base_url)[:2]) + metadata[0].get('href')
    
    return ascii.read(metadata_url)

In [19]:
scifiles = {}

In [22]:
names = ['And '+int_to_roman(i+1)  for i in range(33) if i+1 != 4]

for nm in tqdm.tqdm_notebook(names):
    print(nm)
    if nm in scifiles:
        print('Already done')
    try:
        tab = target(nm, 20*u.arcmin)
        scifiles[nm] = tab
        print('Completed')
    except Exception as e:
        print('failed:\n', repr(e))

And I
Already done
search URL: https://koa.ipac.caltech.edu/cgi-bin/bgServices/nph-bgExec?spt_obj=spatial&locstr=And+I&radunits=arcmin&bgApp=%2FKOA%2Fnph-KOA&calibassoc=assoc&regSize=20.0&resolver=ned&filetype=science&single_multiple=single&instrument_de=deimos

Completed
And II
Already done
search URL: https://koa.ipac.caltech.edu/cgi-bin/bgServices/nph-bgExec?spt_obj=spatial&locstr=And+II&radunits=arcmin&bgApp=%2FKOA%2Fnph-KOA&calibassoc=assoc&regSize=20.0&resolver=ned&filetype=science&single_multiple=single&instrument_de=deimos
Completed
And III
Already done
search URL: https://koa.ipac.caltech.edu/cgi-bin/bgServices/nph-bgExec?spt_obj=spatial&locstr=And+III&radunits=arcmin&bgApp=%2FKOA%2Fnph-KOA&calibassoc=assoc&regSize=20.0&resolver=ned&filetype=science&single_multiple=single&instrument_de=deimos
Completed
And V
Already done
search URL: https://koa.ipac.caltech.edu/cgi-bin/bgServices/nph-bgExec?spt_obj=spatial&locstr=And+V&radunits=arcmin&bgApp=%2FKOA%2Fnph-KOA&calibassoc=assoc&re

In [23]:
for nm, tab in scifiles.items():
    print(nm, len(tab))

And V 84
And I 36
And XXXIII 16
And III 46
And XXX 11
And XI 15
And XXII 28
And XIX 132
And XVI 25
And XXV 5
And XXVI 4
And IX 24
And VI 23
And XVIII 28
And XIV 35
And XXIII 6
And VII 46
And XVII 12
And II 60
And XV 25
And XXIX 38
And VIII 79
And XXVII 28
And XX 5
And X 30
And XII 25
And XXXI 56
And XXVIII 31
And XIII 46
And XXXII 56
And XXI 42
And XXIV 10


In [30]:
merged_table = table.vstack(list(scifiles.values()))

In [41]:
!open .

In [31]:
merged_table

koaid,instrument,targname,imagetyp,frameno,ra,dec,date_obs,ut,elaptime,equinox,progid,proginst,progpi,progtitl,filehand,semid,propmin,prop
str88,str24,str60,str24,int64,float64,float64,str40,str44,float64,float64,str24,str20,str48,str460,str216,str48,int64,str4
DE.20060918.30130.fits,DEIMOS,Andromeda-V-1,object,90,17.56525,47.59606,2006-09-18,08:22:10.29,30.0,2000.0,K238D,KECK,Armandroff,M31s Dwarf Companion Galaxies.,/koadata18/DEIMOS/20060918/lev0/DE.20060918.30130.fits,2006b_k238d,18,f
DE.20060918.30209.fits,DEIMOS,Andromeda-V-1,object,91,17.56554,47.59628,2006-09-18,08:23:29.35,30.0,2000.0,K238D,KECK,Armandroff,M31s Dwarf Companion Galaxies.,/koadata18/DEIMOS/20060918/lev0/DE.20060918.30209.fits,2006b_k238d,18,f
DE.20060918.30358.fits,DEIMOS,Andromeda-V-1,object,92,17.5655,47.59625,2006-09-18,08:25:58.45,1200.0,2000.0,K238D,KECK,Armandroff,M31s Dwarf Companion Galaxies.,/koadata18/DEIMOS/20060918/lev0/DE.20060918.30358.fits,2006b_k238d,18,f
DE.20060918.31672.fits,DEIMOS,Andromeda-V-1,object,93,17.5655,47.59625,2006-09-18,08:47:52.87,1200.0,2000.0,K238D,KECK,Armandroff,M31s Dwarf Companion Galaxies.,/koadata18/DEIMOS/20060918/lev0/DE.20060918.31672.fits,2006b_k238d,18,f
DE.20060918.32954.fits,DEIMOS,Andromeda-V-1,object,94,17.5655,47.59625,2006-09-18,09:09:14.34,1200.0,2000.0,K238D,KECK,Armandroff,M31s Dwarf Companion Galaxies.,/koadata18/DEIMOS/20060918/lev0/DE.20060918.32954.fits,2006b_k238d,18,f
DE.20060918.34230.fits,DEIMOS,Andromeda-V-1,object,95,17.5655,47.59625,2006-09-18,09:30:30.26,1200.0,2000.0,K238D,KECK,Armandroff,M31s Dwarf Companion Galaxies.,/koadata18/DEIMOS/20060918/lev0/DE.20060918.34230.fits,2006b_k238d,18,f
DE.20060918.35590.fits,DEIMOS,Andromeda-V-1,object,96,17.57804,47.60439,2006-09-18,09:53:10.23,20.0,2000.0,K238D,KECK,Armandroff,M31s Dwarf Companion Galaxies.,/koadata18/DEIMOS/20060918/lev0/DE.20060918.35590.fits,2006b_k238d,18,f
DE.20060918.35669.fits,DEIMOS,Andromeda-V-1,object,97,17.57792,47.60436,2006-09-18,09:54:29.68,20.0,2000.0,K238D,KECK,Armandroff,M31s Dwarf Companion Galaxies.,/koadata18/DEIMOS/20060918/lev0/DE.20060918.35669.fits,2006b_k238d,18,f
DE.20060918.37419.fits,DEIMOS,Andromeda-V-1,object,99,17.57792,47.60436,2006-09-18,10:23:39.70,1500.0,2000.0,K238D,KECK,Armandroff,M31s Dwarf Companion Galaxies.,/koadata18/DEIMOS/20060918/lev0/DE.20060918.37419.fits,2006b_k238d,18,f


In [34]:
list(np.unique(merged_table['targname']))

['13buk_S3',
 '16Disk',
 '25Strm',
 '509BlSand18',
 '510BlSand20',
 'A19SmN_________',
 'A19l1',
 'A19l2',
 'A19r1',
 'A21maj',
 'A21min',
 'A23o2',
 'A27Sf2',
 'And16',
 'And17',
 'And19m1',
 'And19m2',
 'And22',
 'And22Mask',
 'And24',
 'And25',
 'And26',
 'And9m1',
 'And9m2',
 'AndVIMask',
 'And_VI',
 'Andromeda-V-1',
 'Andromeda-V-3',
 'Andromeda-V-4',
 'B058-G119',
 'B338-G076',
 'Cas3f3S',
 'Cas3f5S',
 'Cas3f6S',
 'Cas3f7S',
 'Cas3f8S',
 'F228DiH',
 'F233BlS',
 'F234BlS',
 'F235BlS',
 'F603HaS',
 'F702HaS',
 'GC_M31',
 'M31F8_1',
 'M31_GC',
 'M32',
 'M32_3',
 'M32_4',
 'M32_5',
 'M32_HST1',
 'M32_HST2',
 'M32_obs_PA-20',
 'MGC1',
 'Per1f1S',
 'Per1f2S',
 'R11A170_1_PA-97',
 'R11A170_2_PA0',
 'RCB13e_S1',
 'S1600at',
 'SDSS13',
 '_A19DC1________',
 '_A19DC2________',
 '_A19S37________',
 'a19p00',
 'a21p180',
 'a28sdss',
 'a28sdss________',
 'a29_1',
 'a29_2',
 'a29sdss________',
 'and10a',
 'and12',
 'and15',
 'and17p90',
 'and19m45',
 'and19m90',
 'and19p45',
 'and1a',
 'and24',

In [39]:
scifiles['And XXVIII']

koaid,instrument,targname,imagetyp,frameno,ra,dec,date_obs,ut,elaptime,equinox,progid,proginst,progpi,progtitl,filehand,semid,propmin,prop
str22,str6,str15,str6,int64,float64,float64,str10,str11,float64,float64,str5,str5,str6,str91,str54,str11,int64,str1
DE.20110927.20863.fits,DEIMOS,a28sdss,object,72,338.09954,31.26497,2011-09-27,05:47:43.75,20.0,2000.0,U080D,UCLA,M.Rich,Z-PandAS: Exploring the M31 Northern Stream and Dwarf Spheroidals,/koadata18/DEIMOS/20110927/lev0/DE.20110927.20863.fits,2011b_u080d,18,f
DE.20110927.21034.fits,DEIMOS,a28sdss,object,73,338.09954,31.26497,2011-09-27,05:50:34.25,20.0,2000.0,U080D,UCLA,M.Rich,Z-PandAS: Exploring the M31 Northern Stream and Dwarf Spheroidals,/koadata18/DEIMOS/20110927/lev0/DE.20110927.21034.fits,2011b_u080d,18,f
DE.20110927.21198.fits,DEIMOS,a28sdss,object,74,338.09454,31.26425,2011-09-27,05:53:18.81,20.0,2000.0,U080D,UCLA,M.Rich,Z-PandAS: Exploring the M31 Northern Stream and Dwarf Spheroidals,/koadata18/DEIMOS/20110927/lev0/DE.20110927.21198.fits,2011b_u080d,18,f
DE.20110927.21336.fits,DEIMOS,a28sdss,object,75,338.09454,31.26425,2011-09-27,05:55:36.56,20.0,2000.0,U080D,UCLA,M.Rich,Z-PandAS: Exploring the M31 Northern Stream and Dwarf Spheroidals,/koadata18/DEIMOS/20110927/lev0/DE.20110927.21336.fits,2011b_u080d,18,f
DE.20110927.21623.fits,DEIMOS,a28sdss,object,76,338.10083,31.26672,2011-09-27,06:00:23.16,20.0,2000.0,U080D,UCLA,M.Rich,Z-PandAS: Exploring the M31 Northern Stream and Dwarf Spheroidals,/koadata18/DEIMOS/20110927/lev0/DE.20110927.21623.fits,2011b_u080d,18,f
DE.20110927.21772.fits,DEIMOS,a28sdss,object,77,338.10083,31.26672,2011-09-27,06:02:52.71,20.0,2000.0,U080D,UCLA,M.Rich,Z-PandAS: Exploring the M31 Northern Stream and Dwarf Spheroidals,/koadata18/DEIMOS/20110927/lev0/DE.20110927.21772.fits,2011b_u080d,18,f
DE.20110927.22014.fits,DEIMOS,a28sdss,object,78,338.10083,31.26672,2011-09-27,06:06:54.71,8.0,2000.0,U080D,UCLA,M.Rich,Z-PandAS: Exploring the M31 Northern Stream and Dwarf Spheroidals,/koadata18/DEIMOS/20110927/lev0/DE.20110927.22014.fits,2011b_u080d,18,f
DE.20110927.22114.fits,DEIMOS,a28sdss,object,79,338.10096,31.26494,2011-09-27,06:08:34.96,20.0,2000.0,U080D,UCLA,M.Rich,Z-PandAS: Exploring the M31 Northern Stream and Dwarf Spheroidals,/koadata18/DEIMOS/20110927/lev0/DE.20110927.22114.fits,2011b_u080d,18,f
DE.20120915.18823.fits,DEIMOS,d28_3,object,56,338.22063,31.21456,2012-09-15,05:13:43.66,20.0,2000.0,Y213D,CIT-Y,Geha,Precision Abundances of M31 dSph Satellites,/koadata18/DEIMOS/20120915/lev0/DE.20120915.18823.fits,2012b_y213d,18,f
